# Results

Import packages.

In [ ]:
import pandas as pd
from pathlib import Path

Specify the configuration to read results for.

In [ ]:
# * Model choice
alias = "SLSQPEnsemble_3-models_chronos-flowstate-timesfm_opt-mae_1-windows"

# * Random metric
# alias = "SLSQPEnsemble_3-models_moirai-sundial-toto_opt-random_1-windows"

# * Additional models
# alias = "SLSQPEnsemble_4-models_chronos-moirai-sundial-toto_opt-mae_1-windows"

Read all of the configuration's CSV files into a single DataFrame and save the aggregated results.

In [ ]:
input_path = Path(f"../results/{alias}")
csv_files = list(input_path.rglob("*.csv"))

df = pd.concat((pd.read_csv(f).tail(1) for f in csv_files), ignore_index=True)
print(f"df shape: {df.shape}")

output_path = Path(f"../results/all_results/{alias}") / "all_results.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)
print(f"Saved aggregatted results to {output_path}")

df.head()

df shape: (24, 15)
Saved aggregatted results to ../results/all_results/SLSQPEnsemble_3-models_moirai-sundial-toto_opt-random_1-windows/all_results.csv


,dataset,model,eval_metrics/MSE[mean],eval_metrics/MSE[0.5],eval_metrics/MAE[0.5],eval_metrics/MASE[0.5],eval_metrics/MAPE[0.5],eval_metrics/sMAPE[0.5],eval_metrics/MSIS,eval_metrics/RMSE[mean],eval_metrics/NRMSE[mean],eval_metrics/ND[0.5],eval_metrics/mean_weighted_sum_quantile_loss,domain,num_variates
0,us_births/M/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,4.410284e+07,4.410284e+07,5426.967325,0.613907,0.016730,0.016926,4.249311,6640.997002,0.020627,0.016856,0.012764,Healthcare,1
1,us_births/W/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,1.670029e+06,1.670029e+06,990.722241,0.901148,0.013476,0.013526,6.990632,1292.295979,0.017543,0.013449,0.010871,Healthcare,1
2,us_births/D/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,1.672966e+05,1.672966e+05,242.106004,0.356472,0.023300,0.022961,3.736674,409.019051,0.038342,0.022695,0.018589,Healthcare,1
3,bizitobs_service/10S/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,2.538134e+03,2.538134e+03,19.824362,0.770288,0.052069,0.042263,6.532468,50.379898,0.037323,0.014687,0.011562,Web/CloudOps,2
4,bitbrains_rnd/H/short,SLSQPEnsemble_3-models_moirai-sundial-toto_opt...,1.845975e+06,1.845975e+06,172.650594,5.825690,2.346536,0.554711,196.350106,1358.666520,5.946189,0.755603,0.633505,Web/CloudOps,2


View the number of datasets for each term.

In [ ]:
num_short_datasets = df['dataset'].str.contains('short', case=False, na=False).sum()
print(f"Number of short datasets: {num_short_datasets}")

Number of short datasets: 24


View the number of datasets for each domain.

In [ ]:
domain_counts = (
    df.groupby("domain")
      .size()
      .reset_index(name="num_evaluated_datasets")
)

display(domain_counts)

,domain,num_evaluated_datasets
0,Energy,2
1,Healthcare,5
2,Sales,4
3,Transport,7
4,Web/CloudOps,6


Display all of the results and only show the MASE and CRPS columns.

In [ ]:
pd.set_option('display.max_rows', None)

df = df.rename(columns={
    "eval_metrics/MASE[0.5]": "MASE",
    "eval_metrics/mean_weighted_sum_quantile_loss": "CRPS",
},
errors="ignore",
)

keep_cols = [
    "dataset",
    "MASE",
    "CRPS",
    "domain",
    "num_variates",
]
df = df[keep_cols].sort_values(by="CRPS", ascending=False).reset_index(drop=True)
display(df)

,dataset,MASE,CRPS,domain,num_variates
0,car_parts/M/short,0.815056,0.936593,Sales,1
1,hierarchical_sales/D/short,0.789432,0.649840,Sales,1
2,bitbrains_rnd/H/short,5.825690,0.633505,Web/CloudOps,2
3,bitbrains_fast_storage/H/short,1.001568,0.547587,Web/CloudOps,2
4,hierarchical_sales/W/short,0.728154,0.348017,Sales,1
5,ett1/D/short,1.686748,0.289073,Energy,7
6,restaurant/D/short,0.690238,0.264841,Sales,1
7,bizitobs_l2c/H/short,0.464845,0.206205,Web/CloudOps,7
8,sz_taxi/15T/short,0.543895,0.201370,Transport,1
9,sz_taxi/H/short,0.560181,0.135060,Transport,1
